This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
from api_keys import QUANDL  # private key stashed in separate file
API_KEY = QUANDL

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import json
from collections import namedtuple

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
Q_URL = "https://www.quandl.com/api/v3/datasets/"
method = 'FSE/AFX_X/data.json'
params = {'start_date':'2019-10-14', 'end_date':'2019-10-14', 'order':'asc', 'api_key':API_KEY}
query = Q_URL + method

response = requests.get(url=query, params=params)

#print(response.status_code, type(response.json()))

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

#response_json = response.json()
#response_json.keys()

In [5]:
#response_json['dataset_data'].keys()

In [6]:
#response_json['dataset_data']['column_names']

In [7]:
#response_json['dataset_data']['data']

In [8]:
#list(zip(response_json['dataset_data']['column_names'], response_json['dataset_data']['data'][0]))

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

### 1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 


In [9]:
params['start_date'] = '2017-01-01'
params['end_date'] = '2017-12-31'

response = requests.get(query, params)
#print(response.status_code)

### 2. Convert the returned JSON object into a Python dictionary.

In [10]:
afx_json = response.json()

In [11]:
# make an ordered list of the days (could also have used an OrderedDict, but harder to build that)
days = [day[0] for day in afx_json['dataset_data']['data']]

# store each day as a dict key, with Quote namedtuples as values
Quote = namedtuple('Quote', ['open', 'high', 'low', 'close', 'vol'])
afx_dict = {day[0]: Quote(day[1], day[2], day[3], day[4], day[6])
                    for day in afx_json['dataset_data']['data']}
#len(afx_dict)

In [12]:
#afx_dict[days[0]]

### 3. Calculate what the highest and lowest opening prices were for the stock in this period.

In [13]:
low_open = min(quote.open for quote in afx_dict.values() if quote.open)
high_open = max(quote.open for quote in afx_dict.values() if quote.open)

print(f'Highest open: {high_open}\nLowest open: {low_open}')

Highest open: 53.11
Lowest open: 34.0


### 4. What was the largest change in any one day (based on High and Low price)?

In [14]:
hi2lo = max(afx_dict.values(), key=lambda x: x.high - x.low)
print(f'The largest intraday range was {round(hi2lo.high - hi2lo.low, 4)}')

The largest intraday range was 2.81


### 5. What was the largest change between any two days (based on Closing Price)?

In [15]:
# iterate over the ordered list of days, comparing consecutive closes
changes = [afx_dict[days[i]].close - afx_dict[days[i+1]].close for i in range(len(days) - 1)]
max_change = max(abs(change) for change in changes)
print(f'The largest change in consecutive days closing prices was {round(max_change, 4)}')

The largest change in consecutive days closing prices was 2.56


### 6. What was the average daily trading volume during this year?

In [16]:
vols = [quote.vol for quote in afx_dict.values() if quote.vol]
print(f'The average daily trading volume for the year was {round(sum(vols) / len(vols), 2)}')

The average daily trading volume for the year was 89124.34


### 7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [17]:
sorted_vols = sorted(vols)
n = len(sorted_vols)
med_vol = sorted_vols[n//2]
if not (n % 2):  # average the 2 most median values if the list length is even
    med_vol = (med_vol + sorted_vols[n//2 - 1]) / 2.0
print(f'The median trading volume for the year was {round(med_vol, 2)}')

The median trading volume for the year was 76286.0
